In [8]:
import os
import json

In [5]:
import os

# Path to ShapeNetCore.v2 and output
shapenet_path = "../data/ShapeNetCore.v2"
output_path = "/path/to/output/txt_files"
os.makedirs(output_path, exist_ok=True)

# Generate file list
splits = ['train', 'test', 'val']  # Adjust as needed
split_ratio = [0.8, 0.1, 0.1]  # Train/Test/Validation Split
all_files = []

print(f"Number of Taxonomies: {len(os.listdir(shapenet_path))}")
num_files = 0
all_files = []

Number of Taxonomies: 56


In [9]:
name_json_path = os.path.join(shapenet_path, "taxonomy.json")
with open(name_json_path, 'r') as f:
    name_json = json.load(f)
print(f"Number of Taxonomies: {len(name_json)}")
print(f"json dict: {name_json}")

Number of Taxonomies: 354
json dict: [{'synsetId': '02691156', 'name': 'airplane,aeroplane,plane', 'children': ['02690373', '02842573', '02867715', '03174079', '03335030', '03595860', '04012084', '04160586', '20000000', '20000001', '20000002'], 'numInstances': 4045}, {'synsetId': '02690373', 'name': 'airliner', 'children': ['03809312', '04583620'], 'numInstances': 1490}, {'synsetId': '03809312', 'name': 'narrowbody aircraft,narrow-body aircraft,narrow-body', 'children': [], 'numInstances': 14}, {'synsetId': '04583620', 'name': 'widebody aircraft,wide-body aircraft,wide-body,twin-aisle airplane', 'children': [], 'numInstances': 26}, {'synsetId': '02842573', 'name': 'biplane', 'children': [], 'numInstances': 10}, {'synsetId': '02867715', 'name': 'bomber', 'children': [], 'numInstances': 178}, {'synsetId': '03174079', 'name': 'delta wing', 'children': [], 'numInstances': 121}, {'synsetId': '03335030', 'name': 'fighter,fighter aircraft,attack aircraft', 'children': [], 'numInstances': 745}

In [16]:
num_files = 0
all_files = []
for taxonomy in os.listdir(shapenet_path):
    taxonomy_path = os.path.join(shapenet_path, taxonomy)
    if not os.path.isdir(taxonomy_path):
        continue
    
    for model in os.listdir(taxonomy_path):
        model_path = os.path.join(taxonomy_path, model)
        if not os.path.isdir(model_path):
            continue
        for model_file in os.listdir(model_path + "/models"):
            if model_file.endswith(".obj"):
                all_files.append(os.path.join(taxonomy, model, model_file))

    # print(f"Taxonomy: {taxonomy} has {len(os.listdir(taxonomy_path))} models")
    for taxonomy_id_json in name_json:
        if taxonomy_id_json['synsetId'] == taxonomy:
            class_name = taxonomy_id_json['name']
            break
    print(f"Class Name: {class_name} has {len(os.listdir(taxonomy_path))} models")
    num_files += len(os.listdir(taxonomy_path)) 
print(f"Total number of files: {num_files}")
print(f"Total number of files: {len(all_files)}")

Class Name: tower has 133 models
Class Name: skateboard has 152 models
Class Name: pot,flowerpot has 602 models
Class Name: loudspeaker,speaker,speaker unit,loudspeaker system,speaker system has 1597 models
Class Name: camera,photographic camera has 113 models
Class Name: car,auto,automobile,machine,motorcar has 3533 models
Class Name: vessel,watercraft has 1939 models
Class Name: earphone,earpiece,headphone,phone has 73 models
Class Name: rocket,projectile has 85 models
Class Name: helmet has 162 models
Class Name: pillow has 96 models
Class Name: jar has 596 models
Class Name: airplane,aeroplane,plane has 4045 models
Class Name: can,tin,tin can has 108 models
Class Name: table has 8436 models
Class Name: remote control,remote has 66 models
Class Name: knife has 424 models
Class Name: birdhouse has 73 models
Class Name: bus,autobus,coach,charabanc,double-decker,jitney,motorbus,motorcoach,omnibus,passenger vehi has 939 models
Class Name: bathtub,bathing tub,bath,tub has 856 models
Clas

In [17]:
all_files[0:10]

['04460130/92e4cc97df1ebe141203ec4ca6ccf208/model_normalized.obj',
 '04460130/f982e86909ed9e7f2c8891dbc3e6988a/model_normalized.obj',
 '04460130/6d43d4642c7a474638cd8eb23853c009/model_normalized.obj',
 '04460130/a0cc96c45bc569d9ebc1e59456a1eaad/model_normalized.obj',
 '04460130/3735b272459f1b15db439ec9067ff33f/model_normalized.obj',
 '04460130/9e72b97a5af45ded7c272b953086dacf/model_normalized.obj',
 '04460130/f14b153ab99da1bb73a921481ee71edc/model_normalized.obj',
 '04460130/8af5e20ddd4ac314d3e8c09edf9de80a/model_normalized.obj',
 '04460130/46a090f0bd14180bf1a48a667d61ced6/model_normalized.obj',
 '04460130/1fc436fd53db26d667ff3be08608de4d/model_normalized.obj']

In [18]:
# split to train, test, val 0.8, 0.1, 0.1
import random
random.shuffle(all_files)
split_idx = [0, int(split_ratio[0] * len(all_files)), int((split_ratio[0] + split_ratio[1]) * len(all_files))]
train_files = all_files[split_idx[0]:split_idx[1]]
test_files = all_files[split_idx[1]:split_idx[2]]
val_files = all_files[split_idx[2]:]

print(f"Train: {len(train_files)} Test: {len(test_files)} Val: {len(val_files)}")

Train: 41977 Test: 5247 Val: 5248


In [20]:
# save to json
split_files = {
    "train": train_files,
    "test": test_files,
    "val": val_files
}
with open(os.path.join(shapenet_path, "split_files.json"), 'w') as f:
    json.dump(split_files, f)

In [21]:
import open3d as o3d

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [32]:
import open3d as o3d

file_path = "/home/xsamplecontrast/xsampleContrastive/data/ShapeNetCore.v2_3d/04530566/ffb0a1240833569b9041e5038dbb1ef6/models/model_normalized.obj"

# Load the mesh
mesh = o3d.io.read_triangle_mesh(file_path, True)

# Check if the mesh is colored
if mesh.has_vertex_colors() or mesh.has_triangle_material_ids():
    print("Mesh has colors or materials.")
else:
    print("Mesh does not have colors or materials.")

# Visualize the mesh
print(mesh)
o3d.visualization.draw_geometries([mesh])

[Open3D INFO] Skipping non-triangle primitive geometry of type: 2
Mesh has colors or materials.
TriangleMesh with 22631 points and 19642 triangles.


In [33]:
import utils.trimesh_pcd as trimesh_pcd

ModuleNotFoundError: No module named 'trimesh'

In [34]:
!pip install trimesh

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
  Obtaining dependency information for trimesh from https://files.pythonhosted.org/packages/28/d0/8852c9f296b5d979a48d764117ff27a96115d74cb6f8ba083744867415a5/trimesh-4.5.3-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 704.8/704.8 kB 3.3 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python -m pip install --upgrade pip


In [ ]:
import utils.trimesh_pcd as trimesh_pcd
file_path = "/home/xsamplecontrast/xsampleContrastive/data/ShapeNetCore.v2_3d/04530566/ffb0a1240833569b9041e5038dbb1ef6/models/model_normalized.obj"
mesh = trimesh_pcd.load(file_path)
mesh.show()

In [37]:
import matplotlib.pyplot as plt
def render_pcd_to_image_trimesh(pcd):
    # Create a scene with the point cloud
    scene = trimesh_pcd.Scene(pcd)

    # Render the scene to an image
    png_data = scene.save_image(resolution=(1024, 1024), visible=True)

    # Convert PNG data to a NumPy array
    image = plt.imread(png_data, format="png")
    return image

In [40]:
import numpy as np
pcd = trimesh_pcd.PointCloud(np.random.rand(1000, 3), colors=[0, 255, 0])
image = render_pcd_to_image_trimesh(pcd)


ImportError: `trimesh.viewer.windowed` requires `pip install "pyglet<2"`

In [39]:
!pip3 install pyglet

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
  Obtaining dependency information for pyglet from https://files.pythonhosted.org/packages/d3/21/6583a53b9910ece5fd6db9590ae88dc24fd0d5ef76f7c6717b8761388fb8/pyglet-2.0.20-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 945.1/945.1 kB 4.6 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python -m pip install --upgrade pip
